# Country → Subbasins → Predictors: Data Prep for WASS2SHydroR

In [15]:
devtools::install_github("kiema97/AGRHYMET-WASS2SHydroR")

Using GitHub PAT from the git credential store.

Skipping install of 'WASS2SHydroR' from a github remote, the SHA1 (daae2af1) has not changed since last install.
  Use `force = TRUE` to force installation



In [1]:
# ==== PARAMETERS (participants only edit this block) ==========================
COUNTRY_CODE <- "BFA" 
PREDICTOR_VARS <- c("PRCP", "SST")  # choose among available folders under predictors/
SELECTED_MODELS <- NULL  # e.g., c("CanCM3","CCSM4") or NULL to use all available
SELECTED_MODELS <- c("CCSM4","CanCM3")
# Where things live (relative to project root)
PATH_COUNTRIES   <- "static/was_contries.shp"   # shapefile with GMI_CNTRY field
PATH_SUBBASINS   <- "static/subbassins.shp"     # shapefile with HYBAS_ID field
PATH_HISTORICAL  <- "data/was_subbassins_data.csv"  # columns: DATE, HYBAS_ID, Q, prcp, evap
PATH_PREDICTORS  <- "D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep"             # expect subfolders: PRCP/, SST/
setwd("D:/CCR_AOS/WASS2SHydroRTraining")
# Optional: performance/speed knobs
N_CORES <- 4#max(1, parallel::detectCores() - 1)

# ==== Libraries ==============================================================
suppressPackageStartupMessages({
  library(dplyr)
  library(readr)
  library(stringr)
  library(purrr)
  library(tidyr)
  library(sf)
  library(lubridate)
})

# WASS2SHydroR is expected to be installed; falls back gracefully if missing
has_wass2s <- requireNamespace("WASS2SHydroR", quietly = TRUE)
if (!has_wass2s) {
  message("NOTE: Package 'WASS2SHydroR' not found. You can still run most steps;\n",
          "the function 'wass2s_prepare_data()' will be called only if available.")
}

# Helper: safe parallel plan (base R's parallel via mclapply on Unix; fall back on lapply on Windows)
.parallel_map <- function(X, FUN, ...){
  if (.Platform$OS.type == "windows") {
    # Simple fallback for Windows notebooks to avoid cluster overhead for trainees
    lapply(X, FUN, ...)
  } else {
    parallel::mclapply(X, FUN, mc.cores = N_CORES, ...)
  }
}

## 1) Select the user's country and find covered subbasins

In [2]:
stopifnot(length(COUNTRY_CODE) == 1, nchar(COUNTRY_CODE) == 3)

# Read shapefiles
a_countries <- sf::st_read(PATH_COUNTRIES, quiet = TRUE) %>% 
  sf::st_make_valid()
a_subs      <- sf::st_read(PATH_SUBBASINS, quiet = TRUE) %>% 
  sf::st_make_valid()

# Ensure same CRS
if (sf::st_crs(a_countries) != sf::st_crs(a_subs)) {
  a_subs <- sf::st_transform(a_subs, sf::st_crs(a_countries))
}

# Filter country
country <- a_countries %>% filter(.data$GMI_CNTRY == COUNTRY_CODE)
if (nrow(country) == 0) stop("No country with GMI_CNTRY == ", COUNTRY_CODE)

# Intersections: subbasins partially or fully covered by the country polygon
inter_idx <- sf::st_intersects(a_subs, country, sparse = TRUE)
sel <- lengths(inter_idx) > 0
subs_sel <- a_subs[sel, ]

# Classify as FULL vs PARTIAL coverage (by area ratio of intersection)
inter_geom <- sf::st_intersection(sf::st_make_valid(subs_sel), sf::st_make_valid(country))
area_sub   <- sf::st_area(subs_sel)
area_int   <- sf::st_area(inter_geom)
cover      <- as.numeric(area_int) / as.numeric(area_sub)

subs_sel$coverage_class <- ifelse(cover >= 0.999, "FULL", "PARTIAL")
subs_sel$coverage_ratio <- cover

HYBAS_IDS <- subs_sel$HYBAS_ID
length(HYBAS_IDS)

Warning message:
"attribute variables are assumed to be spatially constant throughout all geometries"


[1] 10

In [3]:
# Quick table for trainees
subs_sel %>% 
  st_drop_geometry() %>% 
  arrange(desc(coverage_ratio)) %>% 
  select(HYBAS_ID, coverage_class, coverage_ratio) %>% 
  head(20)

,HYBAS_ID,coverage_class,coverage_ratio
,<dbl>,<chr>,<dbl>
1,1040786860,FULL,1.0000000
2,1040709280,PARTIAL,0.9784857
3,1040786690,PARTIAL,0.9448550
4,1040873650,PARTIAL,0.5413947
5,1040709030,PARTIAL,0.4691100
6,1040915990,PARTIAL,0.2868057
7,1040023890,PARTIAL,0.2122455
8,1040722720,PARTIAL,0.1709675
9,1040873640,PARTIAL,0.1455303


## 2) Load historical data (HYPE-simulated Q) for selected subbasins

In [4]:
hist_df <- readr::read_csv(PATH_HISTORICAL, show_col_types = FALSE) %>%
  mutate(DATE = as.Date(.data$DATE)) %>%
  filter(.data$HYBAS_ID %in% HYBAS_IDS) %>%
  arrange(.data$HYBAS_ID, .data$DATE)

# Sanity check
hist_df %>% group_by(HYBAS_ID) %>% summarise(n = n(), .groups = "drop") %>% head(10)

HYBAS_ID,n
<dbl>,<int>
1040023890,16222
1040641670,16222
1040709030,16222
1040709280,16222
1040722720,16222
1040786690,16222
1040786860,16222
1040873640,16222
1040873650,16222


## 3) Catalog available predictor files (PRCP / SST)

In [5]:
# Walk predictor folders and parse filenames like: CanCM3.PRCP.nc, CCSM4.PRCP_f2025.nc, CCSM4.SST.nc
catalog_predictors <- function(base_dir = PATH_PREDICTORS){
  dirs <- list.dirs(base_dir, full.names = TRUE, recursive = FALSE)
  tibble(dir = dirs) %>%
    mutate(var = basename(dir)) %>%
    filter(var %in% c("PRCP","SST")) %>%
    mutate(files = map(dir, ~list.files(.x, pattern = "\\.nc$", full.names = TRUE))) %>%
    tidyr::unnest(files) %>%
    mutate(
      file = files,
      fname = basename(file),
      # Patterns: Model.VAR.nc OR Model.VAR_fYYYY.nc
      model = str_replace(fname, "^([^.]+)\\..*$", "\\1"),
      var_detect = toupper(str_replace(fname, "^[^.]+\\.([A-Za-z]+).*$", "\\1")),
      init_year = ifelse(str_detect(fname, "_f(\\\
\\d{4})"), as.integer(str_replace(fname, ".*_f(\\\
\\d{4}).*", "\\1")), NA_integer_),
      var = ifelse(var_detect %in% c("PRCP","SST"), var_detect, var)
    ) %>%
    select(model, var, init_year, file)
}

pred_catalog <- catalog_predictors()

# Filter by trainee choices
pred_catalog <- pred_catalog %>% filter(var %in% PREDICTOR_VARS)
if (!is.null(SELECTED_MODELS)) {
  pred_catalog <- pred_catalog %>% filter(model %in% SELECTED_MODELS)
}

pred_catalog %>% arrange(var, model, init_year) %>% head(20)

model,var,init_year,file
<chr>,<chr>,<int>,<chr>
CCSM4,PRCP,NA,D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CCSM4.PRCP.nc
CCSM4,PRCP,NA,D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CCSM4.PRCP_f2025.nc
CanCM3,PRCP,NA,D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CanCM3.PRCP.nc


## 4) Convert NetCDF → data frame **per model** and **per subbasin** using `wass2s_prepare_data()`

In [23]:
if (!has_wass2s) {
  stop("'WASS2SHydroR' is required for NetCDF → data frame conversion. Please install it.")
}

# ---- Integration notes -------------------------------------------------------
# We build, for EACH HYBAS_ID, a **list of data.frames**, where each data.frame
# corresponds to ONE climate model (e.g., CanCM3), and contains:
#   HYBAS_ID, DATE, Q, and predictor columns renamed to the generic pattern
#   expected by WASS2SHydroR (prcp_1, prcp_2, ..., sst_1, ...).
#
# 'wass2s_prepare_data()' is assumed to:
#   - take a NetCDF file path and a polygon geometry (subbasin),
#   - return a data.frame with at least 'DATE' and pixel/feature columns.

# Keep only predictor columns (exclude DATE)
.pick_predictor_cols <- function(df){
  setdiff(names(df), c("DATE", "date", "Date"))
}

# Aggregate pixel columns to a single series (mean by default) and name it
.aggregate_pixels <- function(df, var_name){
  px_cols <- .pick_predictor_cols(df)
  if (length(px_cols) == 0) return(NULL)
  out <- df %>% mutate(DATE = as.Date(.data$DATE)) %>%
    mutate(!!var_name := rowMeans(across(all_of(px_cols)), na.rm = TRUE)) %>%
    select(DATE, all_of(var_name))
  out
}

# Extract one variable (PRCP or SST) for one (HYBAS_ID, nc_file)
extract_var_for_subbasin <- function(hybas_id, nc_path, var_name){
  geom <- subs_sel %>% filter(.data$HYBAS_ID == hybas_id) %>% sf::st_geometry()
  if (length(geom) == 0) return(NULL)
  bbox <- sf::st_bbox(geom)[c(4,1,2,3)]
  df_raw <- WASS2SHydroR::wass2s_prepare_data(x = nc_path, bbox = bbox,cell_layout = "wide")
  # If already feature-engineered (e.g., var_name or var_name_1 present), keep it
  ready_cols <- names(df_raw)[names(df_raw) %in% c(var_name, paste0(var_name, "_1"))]
  if (length(ready_cols) >= 1) {
    sel <- c("DATE", ready_cols[1])
    out <- df_raw[, sel, drop = FALSE]
    names(out)[2] <- var_name
    return(out)
  }
  # Else aggregate pixel columns
  .aggregate_pixels(df_raw, var_name)
}

# Build predictors (wide) for ONE model and ONE subbasin
# files_tbl_one_model: rows for a single 'model', columns: model, var, init_year, file
build_predictor_block_for_model <- function(hybas_id, files_tbl_one_model){
  stopifnot(length(unique(files_tbl_one_model$model)) == 1)
  pieces <- vector("list", nrow(files_tbl_one_model))
  for (i in seq_len(nrow(files_tbl_one_model))){
    row <- files_tbl_one_model[i,]
    var_name <- tolower(row$var)  # "prcp" or "sst"
    tmp <- extract_var_for_subbasin(hybas_id, row$file, var_name)
    if (!is.null(tmp)) pieces[[i]] <- tmp
  }
  pieces <- pieces[lengths(pieces) > 0]
  if (length(pieces) == 0) return(NULL)
  Reduce(function(x, y) dplyr::full_join(x, y, by = "DATE"), pieces) %>% arrange(DATE)
}

# Harmonize predictor names to prcp_1, prcp_2, ..., sst_1, ... (per MODEL)
remap_predictor_names <- function(df){
    return(df)
  preds <- setdiff(names(df), c("HYBAS_ID","DATE","Q"))
  prcp_cols <- preds[str_detect(preds, "(^|_)prcp(\b|_)")]
  sst_cols  <- preds[str_detect(preds,  "(^|_)sst(\b|_)")]
  prcp_cols <- sort(prcp_cols)
  sst_cols  <- sort(sst_cols)
  ren <- c(setNames(paste0("prcp_", seq_along(prcp_cols)), prcp_cols),
           setNames(paste0("sst_",  seq_along(sst_cols )),  sst_cols ))
  dplyr::rename(df, !!!ren)
}

# Build the final NESTED TRAINING LIST expected by the new design:
# training_nested[[HYBAS_ID]][[MODEL]] = data.frame(HYBAS_ID, DATE, Q, prcp_1..., sst_1...)
make_training_nested_list <- function(hist, pred_tbl){
  by_hybas <- split(hist, hist$HYBAS_ID)
  models   <- sort(unique(pred_tbl$model))
  res <- .parallel_map(names(by_hybas), function(hid){
    hdf <- by_hybas[[hid]] %>% select(DATE, HYBAS_ID, Q)
    # For each model, assemble its predictors then join with Q
    per_model <- lapply(models, function(m){
      files_m <- pred_tbl %>% filter(model == m)
      pred_m  <- build_predictor_block_for_model(as.integer(hid), files_m)
      if (is.null(pred_m)) return(NULL)
      out <- dplyr::left_join(hdf, pred_m, by = "DATE")
      # Reorder columns
      pred_cols <- setdiff(names(out), c("HYBAS_ID","DATE","Q"))
      out <- out %>% select(HYBAS_ID, DATE, Q, all_of(pred_cols))
      # Remap to prcp_1, sst_1, ... (model-specific)
      out <- remap_predictor_names(out)
      out
    })
    names(per_model) <- models
    # Drop empty models for this HYBAS_ID
    per_model[!vapply(per_model, is.null, logical(1))]
  })
  names(res) <- names(by_hybas)
  # Drop HYBAS_ID with no models
  res <- res[vapply(res, function(x) length(x) > 0, logical(1))]
  res
}

training_nested <- make_training_nested_list(hist_df, pred_catalog)
length(training_nested)

[read] D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CanCM3.PRCP.nc

[read] D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CCSM4.PRCP.nc

[read] D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CCSM4.PRCP_f2025.nc

[read] D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CanCM3.PRCP.nc

[read] D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CCSM4.PRCP.nc

[read] D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CCSM4.PRCP_f2025.nc

[read] D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CanCM3.PRCP.nc

[read] D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CCSM4.PRCP.nc

[read] D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CCSM4.PRCP_f2025.nc

[read] D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CanCM3.PRCP.nc

[read] D:/CCR_AOS/Wass2sHydro-Training_base/predictors/PRCP/Jun-Sep/PRCP/CCSM4.PRCP.nc

[read] D:/

[1] 10

## 5) Quick sanity checks on the nested output

In [26]:
# Print structure: how many HYBAS_ID and how many models per HYBAS_ID
n_hybas <- length(training_nested)
models_per <- vapply(training_nested, length, integer(1))
list(n_hybas = n_hybas, summary_models_per_hybas = summary(models_per))

# Show head of first HYBAS_ID / first MODEL
first_h <- names(training_nested)[1]
if (!is.null(first_h)) {
  first_m <- names(training_nested[[first_h]])[1]
  if (!is.null(first_m)) {
    training_nested[[first_h]][[first_m]] %>% head()
  }
}

$n_hybas
[1] 10

$summary_models_per_hybas
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      2       2       2       2       2       2 


HYBAS_ID,DATE,Q,prcp
<dbl>,<date>,<dbl>,<dbl>
1040023890,1981-01-01,124.057,NA
1040023890,1981-01-02,119.591,NA
1040023890,1981-01-03,115.742,NA
1040023890,1981-01-04,112.177,NA
1040023890,1981-01-05,108.763,NA
1040023890,1981-01-06,105.476,NA


## 6) (Optional) Filter variables or limit counts per model

In [ ]:





```{r country-subbasins, message=FALSE}

```

```{r view-subsel}

```

## 2) Load historical data (HYPE-simulated Q) for selected subbasins

```{r historical}

```

## 3) Catalog available predictor files (PRCP / SST)

```{r catalog}

```

## 4) Convert NetCDF → data frame **per model** and **per subbasin** using `wass2s_prepare_data()`

```{r nc-to-df, message=FALSE}

```

## 5) Quick sanity checks on the nested output

```{r checks}
# Print structure: how many HYBAS_ID and how many models per HYBAS_ID
n_hybas <- length(training_nested)
models_per <- vapply(training_nested, length, integer(1))
list(n_hybas = n_hybas, summary_models_per_hybas = summary(models_per))

# Show head of first HYBAS_ID / first MODEL
first_h <- names(training_nested)[1]
if (!is.null(first_h)) {
  first_m <- names(training_nested[[first_h]])[1]
  if (!is.null(first_m)) {
    training_nested[[first_h]][[first_m]] %>% head()
  }
}
```

## 6) (Optional) Filter variables or limit counts per model

```{r optional-limit}
# Example: keep only first 3 PRCP and first 2 SST columns per model
limit_predictors <- function(df, k_prcp = 3, l_sst = 2){
  preds <- setdiff(names(df), c("HYBAS_ID","DATE","Q"))
  keep <- c(
    preds[str_detect(preds, "^prcp_\")][seq_len(min(sum(str_detect(preds, "^prcp_\")), k_prcp))],
    preds[str_detect(preds,  "^sst_\")][seq_len(min(sum(str_detect(preds,  "^sst_\")), l_sst ))]
  )
  base <- c("HYBAS_ID","DATE","Q")
  intersect(c(base, keep), names(df)) %>% dplyr::select(df, all_of(.))
}

# Apply if desired (disabled by default)
# training_nested <- lapply(training_nested, function(per_model){
#   lapply(per_model, limit_predictors, k_prcp = 3, l_sst = 2)
# })
```

## 7) Save the prepared nested list for modeling

```{r save}
dir.create("outputs", showWarnings = FALSE)
saveRDS(training_nested, file = file.path("outputs", paste0("training_nested_", COUNTRY_CODE, ".rds")))
message("Saved: ", file.path("outputs", paste0("training_nested_", COUNTRY_CODE, ".rds")))
```

## 8) Next step (outside this notebook)

- Pour **chaque sous-bassin**, vous avez maintenant **plusieurs data.frames (un par modèle)**.
- Passez `training_nested[[HYBAS_ID]][[MODEL]]` à vos fonctions WASS2SHydroR.
- Si besoin, vous pouvez concaténer au niveau modèle (empiler des années/féquences) ou faire de l’ensemblage plus tard.


- Pass `training_list` directly to your WASS2SHydroR training / forecasting functions.
- If you need cross-validation or lag engineering, do it on each element of the list.
- To restrict to FULL-coverage subbasins only, filter `subs_sel$HYBAS_ID[subs_sel$coverage_class=="FULL"]` before step 2.

---

### Notes & Design Choices (for instructors)

- **Zero/low-code for trainees**: they change only `COUNTRY_CODE`, `PREDICTOR_VARS`, optionally `SELECTED_MODELS`.
- **Model provenance in column names**: during assembly, we keep model tags (e.g., `prcp_CanCM3`) so you can compare models. Later we optionally remap to the strict `prcp_1`, `sst_1` pattern if required by WASS2SHydroR.
- **Partial vs full coverage**: we compute area-based coverage ratios to help quality control.
- **Parallelization**: a simple, notebook-friendly parallel helper is included; Windows falls back to sequential to avoid trainee issues.
- **Adaptation hook**: if `wass2s_prepare_data()` already returns features (not pixels), the wrapper picks them; otherwise it aggregates pixel columns (mean). Replace `.aggregate_pixels()` if you prefer median, PCA, quantiles, etc.
